# Marketing Data set: Expoloratory Data Analysis Part 1: User Web Analytics


* Dummy marketing dataset from tableau maketing dashboard link: https://app.datacamp.com/learn/courses/marketing-analytics-in-tableau

* This book here will focus on exploring the web analytics dataset


* As a basic sense check it  will include:
    * Max date
    * Min date 
    * Date granularity
    * Metrics 
    * Dimensions & commentary to purpose or info to extract
    * General feasibility of the data set

    

In [1]:
#Read in packages
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.stats as st
import pingouin as pg
from dotenv import load_dotenv
import os
import sqlalchemy as sa
import psycopg2 as p2
import missingno as msno

In [2]:
# function to stage database ready to receive cleaned data
def create_engine(db_name):
    """create engine for database set as global variable engine and global variable this year"""
    global engine
    POSTGRES_URI = os.getenv('bosh_db')
    password = POSTGRES_URI+db_name
    # return engine for sqlclchemy
    engine = sa.create_engine(password)
    return engine
    

In [3]:
#create engine marketing_bosh dash
engine = create_engine('bosh_marketing')

In [4]:
#read in files from directory 'User_Data.xlsx', 'PaidSocial_Data.xlsx', 'PaidSearch_Data.xlsx', 'Page_Data.xlsx':
#data provided from datacamp course https://app.datacamp.com/learn/courses/marketing-analytics-in-tableau
users_raw = pd.read_excel('User_Data.xlsx')
pdsocial_raw = pd.read_excel('PaidSocial_Data.xlsx')
pdsearch_raw = pd.read_excel('PaidSearch_Data.xlsx')
pages_raw = pd.read_excel('Page_Data.xlsx')

### Marketing ecosystem reporting general break down

##### Marketing metrics usually come a variety of sources. The below section will breifly outline what they are, & how or when they are integrated into a global business intelligence source of truth i.e. a Data Visulisation Platform

* As a general rule, marketing data is the messiest data there is, because unlike product data that appears as rendered key value pairs entered by developers, the frquency of campaign deployments makes them subject to poor data integrity because all dimension data must be manually configured. 
* To track cross platforms careful considersation needs to be paid to naming convenion of campaign heirachies and tracking parameters to ensure that matches between different source platforms can be made
* You won't know what metrics to include without talking to marketing team first. 
* The marketing/media team will instruct you on how to break out the taxonomy entered into the campaign heirachies in order to create the reporting segments they want. 
* Marketing campaigns should be based on business objectives and describe the audience and intended call to action that the campaign is designed to aquire. 
* Digial campaigns are often based off a busineses's website architecure (url path sections) as it forms the basis of how the customer interacts with the business, if it its not tagged it can't be segmented.
* Rule of thumb general business communication to it's customers can be made by segmenting analytics to the url path 3 or path 4 depending on the site content detail, as segments will not be worth persuing unless they are sufficent in volume,url paths 3 and 4 are inclusive of the individual page paths. The site path captures the interest of the user/customer whilst the action/event will signal their intent on that topic
* You can't, or really you SHOULD NOT guess & make inferences about marketing data, or site analytics data in general, just by looking at the account structure, talk to your media team to understand why campaigns were set up in certain way and your web developer team to understand how the taggin was configured. You will not be able to make sense of anything without a data dictionary- especially for web anayltics if you youself do not have access to the tag configuration, you won't understand what the metrics actually mean, they are customized to capture different events.


#### AdServers (Must have source of Truth)

* Marketing metrics from ad servers, are segmented by two types:
    * Cost Metrics - related to budget of campaign bought on CPC or CPM model usually
        * Cost
        * Impressions
        * Cllicks
        * Reach
    * Conversion metrics - usually pixels embeded on site that signal and capture a specific desired action, determined by conversion filtered to a conversion tag name

* For businesses, generally only cost metrics from Ads platforms (Cost & Impressions always, sometimes clicks) are considered a source of truth for a business. 
* Whilst Ads platform conversion metrics are generally not considered a source of truth for the business they are imperative for media teams to have in their campaign monitoring dashboards because in platform conversions are the only metric that media teams can optimize towards. Business source of truth metrics from salesforce/web anayltics only serve as a campaign objective, but in platform metrics are required as marker for campaign optimisiation.
* Cost, impressions, clicks are mutally exclusive and can be added up, but Reach is a distinct count of user id and will change depending on how you segment. Since the actual ids are not given to you just the count of the them, you don't know how to de dupe across days, or placments/creatives 

#### DSPs  (Must have source of Truth if running)

* Most common DSP is DV360 
* Marketing metrics from DSPs, are segmented by two types:
    * Cost Metrics - related to budget of campaign bought on CPC or CPM model usually
        * Cost
        * Impressions
        * Cllicks
    * Conversion metrics - usually pixels embeded on site that signal and capture a specific desired action, determined by conversion filtered to a conversion tag name
    * Appears auto tagged in in Google anayltics as source dbm
    * Different campain structure
        * DV360 IO = CM Camapaign 
        * DV360 line item - houses targeting stratgey sits below campaign but above placement IO : Line Item = 1:Many - Thus unfortunately 
        * DV360 creative = CM mediabuy Line Item: DV360 Creative = Many: Many
        * DV360 does not house the actual creatives 
       

##### Campaign manager <br/>


* For large companies and agencies, ads served in other platforms will also be trafficked in Google CM - the most commonly used Ads Managers
* Trafficking entails that the same campaign structure that exsits in an external ads platform such as Facebook, or Tiktok is also created in Campaign Manager in such a way that the data model would be 
    * Social Sources
        * Social Campaign -> CM Campaign
        * Social Adset/Placment -> Mediabuy Name
        * Creative -> Creative
        * Creatives are housed in Native Social platform
    * Search Sources
        * Adwords campaigns usually have to manaully trafficked
        * However as a rule of thumb, businesses that can afford CM can afford SA360 - that you can sync and manage Adwords/Google Ads in, so you can just enable the 360 auto tracking feature in CM 
            * SA360 appears in CM under the campaign name DART SEARCH, SA360 Accounts get moved down to media buy, you have to manually traffic in search campaign under creative, sometimes for convenience people just apply a percentage mulitplitive from account to campaigns if campaigns are not trafficked but that is just an estimtor as the actual granualrity does not exist. 
            * No creatives housed
    * Direct buys with publishers 
        * CM houses actual physical creatives under creative heiarchy 


* The purpose of CM is to create a campaign audit and reconcile what was booked with what was delivered from the publisher's platform (Ad plaform, DFP and or DSPs) to ensure accuracy
    * Ad ops team will create CM click & impressions trackers (floodlights) in DCM and give them to Ad ops team to place in platform usually seen as 1x1 trackers.
    * The impressions and click trackers allow CM to break down conversions by post impression and post click. Total conversions sums the two, but depending on the business, you post click and post impression broken out. 
    * The advantage of CM it accounts for impression tracking (conversions made from serving (seeing) the ad but not clicking on the link) which web analytics cannot account for.
    * Although click through conversions are generally accepted as showing gretaer intent, post impression conversions can be useful to understand campaigns who's purpose is to generate awareness by targeting page lands
    * Conversions have recongised look back windows to when to stop attributing to conversion the campaign, default is generally 90 days but you can always custom configure



##### Web & App Analytics  <br/>

* Depending on the business type analyics can be used to track different things, Ad Manager trackers, Ad platform trackers e.g. facebook pixel need to be given to the Web developer to implement on the website/app to make sure things site conversions are tracked
* It is almost always the source of truth for visits, and marketing leads (sign ups)
* If the site is ecommerce it can also be the source of truth for revenue
* Depending on the business it is also of often the primary generator of click stream data, though some businesses like to process their data for click stream data in mixpanel or amplitude etc
* Campaign activity is tracked by appending the relevent utm parameters and campaign heiarchy values to the landing page url of ads
    * CM & SA360 and Google Ads are all auto tageed in Google Anlaytics as DFA, Dart 

* Analytics data is based on sessions - usually 30 min window from openinning to no activity, or till the application is closed
* Session data feeds up into cookie ID which is unique tracker, usually based of device ID, that acts a proxy for a user, also expires in 90 days
* User id is the registered id generated for user who has signed up to your business, usually idenfited by a email or phone number depending on how they signed up, a user can have multp 
* Reports are pulled with last click attribution, so dimension attriubted to a metric will be whatever last product/segment/feature they interacted with, so the time segment of reports is important, depending on the type of activity... e.g. for streaming platforms it might better to have hourly data, because users are likely to browse more than 1 show (video plays) during a session, same for ecom depending on complexity. Pulling by day will show you the last touch page for the customer that day for a specfic action/interaction. 

#### Salesforce Core/CRMs/Marketo  <br/>

* Depending on the business type, business that are generally B2B, utilize salesforce or marketo to create marketing campaign segments (develop personas and marketing messaging) from customer profiles based of demographics, customer life time value recognised reveneue, and sometimes other campaigns/product features they engaged with
* Commmon salesforce core (sales and or service cloud) metrics include:
    * opportunies
    * leads
    * engagement
    * sales
    * revenue
    * call
* Metrics are broken out by business units for large enough businesses and individual sales agents, the campaign (whatever pitch they are pitching) and the contact details and interactions logs of leads and aquired accounts
* Similar to ads platform data, everything has to be manually entered by a sales agent, and attributed to the right campaign so it's important naming conventions are considered here to map it to back to the marketing leads 
* What metrics you track depends on the campaign objetive that was breifed in, the tpye of business and where you sit in sales funnel 



 




#### Basic Ads Platfor data

* A quick glance at the ads data shows that it has landing page, which generally comes from social under the dimension page url
* None of these look to be oringal source data but a made up data model, because they lack campaign heirarchy dimensions and their respective keys


In [5]:
# preview social
pdsocial_raw.head()

,Ad Group,Demographic,Platform,Served,Landing Page,Creative,Spend,Date,Impressions,Clicks,Visits,Engaged Visits,Purchases,Account Follows,New Customers
0,Shop - Desktop,"Female, 18 - 30, Follows Fashion",Facebook,In Stories,Category Page: Necklaces,Necklace Stack on Influencer,523,2022-09-01,840.000000,42,42,40,0,33,0
1,Shop - Desktop,"Female, 18 - 30, Follows Fashion",Facebook,In Stories,Category Page: Rings,Ring Stack on Influencer,593,2022-12-14,800.000000,48,48,46,3,36,3
2,Shop - Mobile,"Female, 18 - 30, Follows Fashion",Instagram,In Feed,Category Page: Rings,Ring Stack on Influencer,518,2022-12-21,375.000000,45,44,42,3,38,3
3,Shop - Mobile,"Female, 18 - 30, Follows Fashion",Instagram,In Feed,Category Page: Earrings,Up Close: Earring Collage on Model,510,2022-10-14,336.363636,37,37,29,1,8,1
4,Shop - Mobile,"Female, 18 - 30, Follows Fashion",Instagram,In Feed,Category Page: Earrings,Up Close: Earring Collage on Model,514,2022-11-27,411.111111,37,37,29,4,8,4


In [6]:
#social landing page to link with users data
pdsocial_raw["Landing Page"].value_counts()

Landing Page
Category Page: Earrings     43
Homepage                    39
Category Page: Rings        34
Category Page: Bracelets    34
Category Page: Necklaces    28
Name: count, dtype: int64

In [7]:
# preview search
pdsearch_raw.head()

,Ad Group,Landing Page,Ad Copy,Spend,Date,Impressions,Impression Share,Clicks,Visits,Engaged Visits,Purchases,New Customers
0,Phrase - Mobile - Trendy Affordable Jewelry,Homepage,JewelryCo: Get the Hottest Jewelry Trends This...,555,2022-07-01,740,0.13,37,37,0,0,0
1,Exact - Desktop - Small Diamond Stud Earrings,Category Page: Earrings,Itty Bitty diamond studs? Say no more,540,2022-07-02,900,0.19,36,33,5,3,3
2,Phrase - Desktop - Thin Rings,Category Page: Rings,Emphemeral Ring Vibes - New Each Week,574,2022-07-03,984,0.16,41,38,1,1,1
3,Phrase - Mobile - Cheap Cute Jewelry,Homepage,JewelryCo: Get the Hottest Jewelry Trends This...,480,2022-07-04,416,0.23,32,32,3,2,2
4,Phrase - Desktop - Affordable Beaded Bracelets,Category Page: Bracelets,Summer Arm Candy Need an Update? Check our our...,585,2022-07-05,702,0.19,39,35,1,1,1


In [8]:
# understanding seaarch landing page
pdsearch_raw["Landing Page"].value_counts()

Landing Page
Category Page: Rings        48
Category Page: Necklaces    38
Homepage                    31
Category Page: Bracelets    31
Category Page: Earrings     30
Name: count, dtype: int64

In [9]:
users_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33013 entries, 0 to 33012
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Cookie ID          33013 non-null  int64         
 1   Session ID         33013 non-null  int64         
 2   User ID            14106 non-null  float64       
 3   Geography          14106 non-null  object        
 4   Email              33013 non-null  object        
 5   Session Date       33013 non-null  datetime64[ns]
 6   Channel            33013 non-null  object        
 7   Landing Page Type  33013 non-null  object        
 8   Final Page Type    33013 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 2.3+ MB


In [10]:
users_raw.head()

,Cookie ID,Session ID,User ID,Geography,Email,Session Date,Channel,Landing Page Type,Final Page Type
0,11115663,939901008,4.967845e+09,Europe,35248@gmail.com,2022-10-18,Organic,Homepage,Category Page
1,11119899,575263040,8.313226e+09,North America,74328@gmail.com,2022-11-19,Organic,Category Page,Cart
2,11120993,148001144,5.084898e+09,North America,13890@gmail.com,2022-10-29,Organic,Category Page,Category Page
3,11155419,306657377,NaN,NaN,,2022-11-18,Paid Social,Homepage,Category Page
4,11162830,825717398,NaN,NaN,,2022-10-26,Paid Social,Homepage,Product Page


### Users data set

* The users has a total of 33,012 rows

* User ID and Geography have the same number of rows 14,106 and make up less then 50% of the data set

*• It is interesting that Geography is not autofilled by IP address, thus inferring that it is extrpolated from the user manually inputting their addresses

* Email although at first glance has no missing fields by looking at the df info, the missing fields do not appear as NA but are blank spaces, and look to match missing user ID and geography

* The data is not time stamped but has a date, and provides both the landing page & the final page of a unique session on a particular date

* We want to clean the make the missing User ID, Geography, and email values = Visitor 

### makerting to web analytics merge 

* Web Analytics data is usually joined to marketing data based on campaign info, at different granularities

* utm_source = ads platform name e.g. Meta Ads

* utm_medium is a free text field, which auto pouplates adwords data with cpc, since adwords is predominately search. You could instead explore using manually entering this accross channels so that it displays the tactic e.g. for adwords you have utm_medium differenitate between shopping, brand, generic, RLSA, youtube ect, vs Feeds or stories in social, but this would be harder since these are usually in the adset level in social but medium sits about campaign. 

* utm_campaign is the campaign_name for google analytics, it auto tags utm_campaigns from adwords as search campaign, however this more reflective of the targeting level of detail in NON search ad platforms ie. mediabuys/placements/adset

* utm_content is a free text field. you can use to popluate NON search campaign mediabuy/placement/adset names so you can get the tartegting e.g. audience demographics, look-a-like, geo, interests etc, 

* google ads does not not auto tagg utm_content, so you use it to describe the ad copy shown such that it matches the general creative description of your non search campaigns, usually useed to describe how ad appears in search. Since Non search campaigns are primarily visual based pictures, you can use the utm_campaign name of non seach campagins to describe the campaign, or alternatively it can be a field in the meidabuy/placement/adset name taxnomy. 

* The dummy web dataset we have does not have utm campagin, thus it can only be joined to the dummy marketing data by date, channel and landing page. Thus the lowest level of granularity is the landing page for a channel per day

 

In [11]:
def unique_vals(df, cols):
    """for columns in df print how many unique values in cols"""
    for i in cols:
        n = df[i].nunique()
        print(f'{i} has {n} unique values')

In [12]:
# number of unique values in users_raw df
unique_vals(users_raw, ["Cookie ID", "Session ID", "User ID", 'Channel', "Landing Page Type", "Final Page Type"])

Cookie ID has 18086 unique values
Session ID has 33013 unique values
User ID has 7588 unique values
Channel has 4 unique values
Landing Page Type has 2 unique values
Final Page Type has 5 unique values


In [13]:
def clean_col_names(df):
    """renames all cols in df to be lower and no spaces"""
    df.columns = [i.lower().replace(' ', "_") for i in df.columns]
    return df

In [14]:
# clean the column names to be all lower case with underscores in place of spacing for easier input into the database
users_raw2 = clean_col_names(users_raw)

In [15]:
# pandas has changed where you can't have NA values for cols anymore so we will need to seperate
# by 
users_raw_reg1 = users_raw2[users_raw2.user_id.isna() == False]
users_raw_unreg1 = users_raw2[users_raw2.user_id.isna() == True]

In [16]:
# sense check unregistered users df
users_raw_unreg1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18907 entries, 3 to 33012
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cookie_id          18907 non-null  int64         
 1   session_id         18907 non-null  int64         
 2   user_id            0 non-null      float64       
 3   geography          0 non-null      object        
 4   email              18907 non-null  object        
 5   session_date       18907 non-null  datetime64[ns]
 6   channel            18907 non-null  object        
 7   landing_page_type  18907 non-null  object        
 8   final_page_type    18907 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 1.4+ MB


In [17]:
# update the user_id, geography & email cols to say Visitor
users_raw_unreg1.loc[:,'user_id'] = 'Visitor'
users_raw_unreg1.loc[:,'geography'] = 'Visitor'
users_raw_unreg1.loc[:,'email'] = 'Visitor'


/var/folders/jr/slp1kljj397ffkxtw33xbkz40000gn/T/ipykernel_87396/3073643481.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Visitor' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  users_raw_unreg1.loc[:,'user_id'] = 'Visitor'


In [18]:
# preview changes
users_raw_unreg1.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type
3,11155419,306657377,Visitor,Visitor,Visitor,2022-11-18,Paid Social,Homepage,Category Page
4,11162830,825717398,Visitor,Visitor,Visitor,2022-10-26,Paid Social,Homepage,Product Page
5,11233735,994753903,Visitor,Visitor,Visitor,2022-11-16,Paid Social,Homepage,Homepage
6,11250039,339981224,Visitor,Visitor,Visitor,2022-11-15,Paid Social,Category Page,Category Page
7,11254637,794926467,Visitor,Visitor,Visitor,2022-09-19,Paid Social,Homepage,Product Page


In [19]:
# we want to also change the ids to strs

def change_df_col_dtype(df, cols, type):
    """"takes a df, list of cols that need to be convereted to one data type and returns df of those cols converted to that one data type"""
    new_types = dict(zip(cols, [type for i in range(len(cols))]))
    df = df.astype(new_types)
    return df

In [52]:
# make int first then make str
users_raw_unreg = change_df_col_dtype(users_raw_unreg1,['user_id', 'cookie_id', 'session_id'],str)


In [21]:
# check changes to unregiestered to make sure all column data types are correct
users_raw_unreg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18907 entries, 3 to 33012
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cookie_id          18907 non-null  object        
 1   session_id         18907 non-null  object        
 2   user_id            18907 non-null  object        
 3   geography          18907 non-null  object        
 4   email              18907 non-null  object        
 5   session_date       18907 non-null  datetime64[ns]
 6   channel            18907 non-null  object        
 7   landing_page_type  18907 non-null  object        
 8   final_page_type    18907 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 1.4+ MB


In [22]:
users_raw_unreg.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type
3,11155419,306657377,Visitor,Visitor,Visitor,2022-11-18,Paid Social,Homepage,Category Page
4,11162830,825717398,Visitor,Visitor,Visitor,2022-10-26,Paid Social,Homepage,Product Page
5,11233735,994753903,Visitor,Visitor,Visitor,2022-11-16,Paid Social,Homepage,Homepage
6,11250039,339981224,Visitor,Visitor,Visitor,2022-11-15,Paid Social,Category Page,Category Page
7,11254637,794926467,Visitor,Visitor,Visitor,2022-09-19,Paid Social,Homepage,Product Page


In [23]:
users_raw_reg1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14106 entries, 0 to 32981
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cookie_id          14106 non-null  int64         
 1   session_id         14106 non-null  int64         
 2   user_id            14106 non-null  float64       
 3   geography          14106 non-null  object        
 4   email              14106 non-null  object        
 5   session_date       14106 non-null  datetime64[ns]
 6   channel            14106 non-null  object        
 7   landing_page_type  14106 non-null  object        
 8   final_page_type    14106 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 1.1+ MB


In [54]:
# change the reg users cols to int bc user_id was float first
users_raw_reg2 = change_df_col_dtype(users_raw_reg1,['user_id', 'cookie_id', 'session_id'],int)
# change the reg users cols to str
users_raw_reg = change_df_col_dtype(users_raw_reg2,['user_id', 'cookie_id', 'session_id'],str)

In [55]:
#sense check data has been inputted correctly
users_raw_reg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14106 entries, 0 to 32981
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cookie_id          14106 non-null  object        
 1   session_id         14106 non-null  object        
 2   user_id            14106 non-null  object        
 3   geography          14106 non-null  object        
 4   email              14106 non-null  object        
 5   session_date       14106 non-null  datetime64[ns]
 6   channel            14106 non-null  object        
 7   landing_page_type  14106 non-null  object        
 8   final_page_type    14106 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 1.1+ MB


In [56]:
users_raw_reg.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type
0,11115663,939901008,4967844706,Europe,35248@gmail.com,2022-10-18,Organic,Homepage,Category Page
1,11119899,575263040,8313226354,North America,74328@gmail.com,2022-11-19,Organic,Category Page,Cart
2,11120993,148001144,5084897627,North America,13890@gmail.com,2022-10-29,Organic,Category Page,Category Page
13,11151294,182504424,8118651420,North America,81004@yahoo.com,2022-08-09,Paid Search,Category Page,Category Page
16,11155684,956966094,4376444590,North America,11818@gmail.com,2022-11-22,Paid Social,Homepage,Category Page


In [57]:
# combine registered and unregistered users df
users_clean1 = pd.concat([users_raw_unreg,users_raw_reg])  
users_clean1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33013 entries, 3 to 32981
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cookie_id          33013 non-null  object        
 1   session_id         33013 non-null  object        
 2   user_id            33013 non-null  object        
 3   geography          33013 non-null  object        
 4   email              33013 non-null  object        
 5   session_date       33013 non-null  datetime64[ns]
 6   channel            33013 non-null  object        
 7   landing_page_type  33013 non-null  object        
 8   final_page_type    33013 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 2.5+ MB


In [59]:
users_clean1.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type
3,11155419,306657377,Visitor,Visitor,Visitor,2022-11-18,Paid Social,Homepage,Category Page
4,11162830,825717398,Visitor,Visitor,Visitor,2022-10-26,Paid Social,Homepage,Product Page
5,11233735,994753903,Visitor,Visitor,Visitor,2022-11-16,Paid Social,Homepage,Homepage
6,11250039,339981224,Visitor,Visitor,Visitor,2022-11-15,Paid Social,Category Page,Category Page
7,11254637,794926467,Visitor,Visitor,Visitor,2022-09-19,Paid Social,Homepage,Product Page


In [60]:
# create year, month and day field of session date
users_clean1.loc[:, 'session_date_month'] = users_clean1['session_date'].dt.month

In [61]:
# number of unique vistiors based on Cookies that not created an login
users_clean1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33013 entries, 3 to 32981
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   cookie_id           33013 non-null  object        
 1   session_id          33013 non-null  object        
 2   user_id             33013 non-null  object        
 3   geography           33013 non-null  object        
 4   email               33013 non-null  object        
 5   session_date        33013 non-null  datetime64[ns]
 6   channel             33013 non-null  object        
 7   landing_page_type   33013 non-null  object        
 8   final_page_type     33013 non-null  object        
 9   session_date_month  33013 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(8)
memory usage: 2.6+ MB


In [62]:
users_clean1.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type,session_date_month
3,11155419,306657377,Visitor,Visitor,Visitor,2022-11-18,Paid Social,Homepage,Category Page,11
4,11162830,825717398,Visitor,Visitor,Visitor,2022-10-26,Paid Social,Homepage,Product Page,10
5,11233735,994753903,Visitor,Visitor,Visitor,2022-11-16,Paid Social,Homepage,Homepage,11
6,11250039,339981224,Visitor,Visitor,Visitor,2022-11-15,Paid Social,Category Page,Category Page,11
7,11254637,794926467,Visitor,Visitor,Visitor,2022-09-19,Paid Social,Homepage,Product Page,9


In [63]:
# see how many user ids and the cookie ids differ by month
reg_users_cookie_avg = users_clean1[users_clean1.user_id != 'Visitor'].groupby('session_date_month').agg(users=('user_id', 'nunique')
                                                                                  ,cookies=('cookie_id', 'nunique')
                                                                                   ).reset_index()

reg_users_cookie_avg

,session_date_month,users,cookies
0,7,1976,1994
1,8,2022,2034
2,9,1957,1969
3,10,1997,2013
4,11,1994,2006
5,12,1691,1697


In [64]:
# ratio of cookies to a user id
reg_users_cookie_avg.loc[:, 'pct_user_to_cookie'] = reg_users_cookie_avg.loc[:, 'cookies']/reg_users_cookie_avg.loc[:, 'users']
reg_users_cookie_avg

,session_date_month,users,cookies,pct_user_to_cookie
0,7,1976,1994,1.009109
1,8,2022,2034,1.005935
2,9,1957,1969,1.006132
3,10,1997,2013,1.008012
4,11,1994,2006,1.006018
5,12,1691,1697,1.003548


In [65]:
# average cookie to user id diff is 0.6% in the entire data set
reg_users_cookie_avg['pct_user_to_cookie'].mean()

np.float64(1.0064590221342182)

• Most users segemented by month only have 1 cookie to their user_id, the difference at most is 0.9% so we can conservatively assume the number of cookies in the un registered data set can serve as a proxy for unregistered user ids

In [66]:

# no missing channels should be utm channel is auto taggeed
users_clean1.channel.value_counts(dropna=False)

channel
Organic        15309
Paid Search     6696
Paid Social     6507
Email           4501
Name: count, dtype: int64

In [67]:
# earliest session date
users_clean1.session_date.min()

Timestamp('2022-07-01 00:00:00')

In [68]:
# latest session date
users_clean1.session_date.max()

Timestamp('2022-12-25 00:00:00')

• Date range for sessios is aprroximately 6 months start of july 2022-07-01 to christmas day 2022-12-25

In [69]:
# last date entry of users
users_clean1.session_date.max()

Timestamp('2022-12-25 00:00:00')

In [70]:
users_clean1['landing_page_type'].value_counts()

landing_page_type
Homepage         22153
Category Page    10860
Name: count, dtype: int64

In [71]:
# for all visitors
users_clean1['final_page_type'].value_counts()

final_page_type
Category Page                 12161
Product Page                   6180
Homepage                       6029
Cart                           6016
Purchase Confirmation Page     2627
Name: count, dtype: int64

In [72]:
# sensse check that only registered users can make purchases & recieve emails review by propotion
users_clean1.loc[users_clean1.user_id == 'Visitor'].groupby('channel')['final_page_type'].value_counts(dropna= False, normalize = True)

channel      final_page_type
Organic      Category Page      0.397338
             Cart               0.203735
             Product Page       0.201053
             Homepage           0.197874
Paid Search  Category Page      0.410023
             Homepage           0.201819
             Product Page       0.196207
             Cart               0.191950
Paid Social  Category Page      0.397059
             Product Page       0.214052
             Homepage           0.196351
             Cart               0.192538
Name: proportion, dtype: float64

In [73]:
# sensse check how many registered users made purchases review by propotion
users_clean1.loc[users_clean1.user_id != 'Visitor'].groupby('channel')['final_page_type'].value_counts(dropna= False, normalize = True)

channel      final_page_type           
Email        Purchase Confirmation Page    0.285048
             Category Page                 0.278827
             Product Page                  0.150855
             Cart                          0.143524
             Homepage                      0.141746
Organic      Category Page                 0.347386
             Product Page                  0.180275
             Homepage                      0.168256
             Cart                          0.167303
             Purchase Confirmation Page    0.136780
Paid Search  Category Page                 0.306283
             Purchase Confirmation Page    0.212696
             Cart                          0.168194
             Homepage                      0.156414
             Product Page                  0.156414
Paid Social  Category Page                 0.366843
             Homepage                      0.181305
             Product Page                  0.173898
             Cart       

In [74]:
# sense check who what content unregistered users arrive on
users_clean1.loc[users_clean1.user_id == 'Visitor'].groupby('channel')['landing_page_type'].value_counts(dropna= False, normalize = True)

channel      landing_page_type
Organic      Homepage             0.669316
             Category Page        0.330684
Paid Search  Homepage             0.659830
             Category Page        0.340170
Paid Social  Homepage             0.672658
             Category Page        0.327342
Name: proportion, dtype: float64

In [75]:
# sense check who what content registered users arrive on
users_clean1.loc[users_clean1.user_id != 'Visitor'].groupby('channel')['landing_page_type'].value_counts(dropna= False, normalize = True)

channel      landing_page_type
Email        Homepage             0.675850
             Category Page        0.324150
Organic      Homepage             0.676459
             Category Page        0.323541
Paid Search  Homepage             0.684555
             Category Page        0.315445
Paid Social  Homepage             0.670547
             Category Page        0.329453
Name: proportion, dtype: float64

In [76]:
#drop session_date_month 
users_clean1.drop(['session_date_month'], axis = 1, inplace=True)
users_clean1.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type
3,11155419,306657377,Visitor,Visitor,Visitor,2022-11-18,Paid Social,Homepage,Category Page
4,11162830,825717398,Visitor,Visitor,Visitor,2022-10-26,Paid Social,Homepage,Product Page
5,11233735,994753903,Visitor,Visitor,Visitor,2022-11-16,Paid Social,Homepage,Homepage
6,11250039,339981224,Visitor,Visitor,Visitor,2022-11-15,Paid Social,Category Page,Category Page
7,11254637,794926467,Visitor,Visitor,Visitor,2022-09-19,Paid Social,Homepage,Product Page


* cookie_ids are 8 long
* session_id are 9 long
* user_id are 10 long

In [79]:
len(users_clean1[users_clean1.user_id != 'Visitor'].iloc[0,2])

10

In [ ]:
len(users_clean1.iloc[0,])

9

• As suspected only registered users can make purchases and be sent EDMs
• We want to create a metric that flags 1 for purchases and 0 for no purchase so we can sum for a session

• Registered users vs non registered have same pslit about 1/3 land on home page and 2/3 category page

User Analytics Artifacts

* Break out into 4 data sets:

    1. Table_name: dates: unique dates and their
        * date DATE PRIMARY KEY NOT NULL
        * event VARCHAR(250)
        * week_commencing_monday DATE NOT NULL
        * day_of_week VARCHAR(250) NOT NULL

    2. Table_name: registered_users table - segment df to only user_id is not visitor - stores user demographics
        * user_id VARCHAR(10) PRIMARY KEY NOT NULL
        * geography VARCHAR(250) NOT NULL
        * email VARCHAR(250) NOT NULL

    3. Table_name sessions data sets - drop the geography, email from data set
        * session_id VARCHAR(8) PRIMARY KEY NOT NULL
        * coookie_id VARCHAR(9) NOT NULL
        * user_id VARCHAR(10) NOT NULL
        * session_date DATE NOT NULL
        * channel VARCHAR(250) NOT NULL
        * landing_page_type VARCHAR(250) NOT NULL
        * final_page_type VARCHAR(250) NOT NULL

    4. Table_name marketing_site_analytics: aggreated site metrics by day for the channel and landing page 
        * date DATE NOT NULL
        * channel VARCHAR(250) NOT NULL
        * landing_page_type VARCHAR(250) NOT NULL
        * final_page_type VARCHAR(250) NOT NULL
        * web_id SERIAL PRIMARY KEY

    




In [118]:
def make_flag(value,flag):
    if value == flag:
        return 1
    else:
        return 0

In [119]:
users_clean1.loc[:, 'purchases'] = users_clean1.loc[:, 'final_page_type'].apply(lambda x : make_flag(x, 'Purchase Confirmation Page') )
users_clean1.loc[:, 'abandon_carts'] = users_clean1.loc[:, 'final_page_type'].apply(lambda x : make_flag(x, 'Cart') )
users_clean1.head()

,cookie_id,session_id,user_id,geography,email,session_date,channel,landing_page_type,final_page_type,session_date_month,session_date_day,session_date_year,purchases,abandon_carts
3,11155419,306657377,Visitor,Visitor,Visitor,2022-11-18,Paid Social,Homepage,Category Page,11,18,2022,0,0
4,11162830,825717398,Visitor,Visitor,Visitor,2022-10-26,Paid Social,Homepage,Product Page,10,26,2022,0,0
5,11233735,994753903,Visitor,Visitor,Visitor,2022-11-16,Paid Social,Homepage,Homepage,11,16,2022,0,0
6,11250039,339981224,Visitor,Visitor,Visitor,2022-11-15,Paid Social,Category Page,Category Page,11,15,2022,0,0
7,11254637,794926467,Visitor,Visitor,Visitor,2022-09-19,Paid Social,Homepage,Product Page,9,19,2022,0,0


In [120]:
users_clean1.final_page_type.value_counts()

final_page_type
Category Page                 12161
Product Page                   6180
Homepage                       6029
Cart                           6016
Purchase Confirmation Page     2627
Name: count, dtype: int64

In [121]:
#total abandon carts sense check
users_clean1['abandon_carts'].sum()

np.int64(6016)

In [122]:
#total abandon carts sense check
users_clean1['purchases'].sum()

np.int64(2627)

In [123]:
users_clean1.geography.value_counts()

geography
Visitor                 18907
North America           10611
Europe                   1426
Middle East & Africa      726
South America             693
Asia & Pacific            650
Name: count, dtype: int64

In [124]:
users_clean1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33013 entries, 3 to 32981
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   cookie_id           33013 non-null  object        
 1   session_id          33013 non-null  object        
 2   user_id             33013 non-null  object        
 3   geography           33013 non-null  object        
 4   email               33013 non-null  object        
 5   session_date        33013 non-null  datetime64[ns]
 6   channel             33013 non-null  object        
 7   landing_page_type   33013 non-null  object        
 8   final_page_type     33013 non-null  object        
 9   session_date_month  33013 non-null  int32         
 10  session_date_day    33013 non-null  int32         
 11  session_date_year   33013 non-null  int32         
 12  purchases           33013 non-null  int64         
 13  abandon_carts       33013 non-null  int64         


In [ ]:
users_clean1.columns

Index(['cookie_id', 'session_id', 'user_id', 'geography', 'email',
       'session_date', 'channel', 'landing_page_type', 'final_page_type',
       'session_date_month', 'session_date_day', 'session_date_year',
       'purchases', 'abandon_carts'],
      dtype='object')

### Users raw data model

• ingest the cleaned users web data into 2 seperate tables registered_users_raw, unregistered_users_raw both have session_id as primary key

• 'cookie_id', 'session_id', 'user_id', 'geography', 'email', 'session_date', 'channel', 'landing_page_type', 'final_page_type' : VARCHAR(250)

• 'session_date_month', 'session_date_day':  NUMERIC(2,0)

• 'session_date_year' :  NUMERIC(4,0)

• 'purchases', 'abandon_carts' : INT()

In [ ]:
def raw_data_to_db(df,table, engine=engine, schema = 'bosh_marketing', if_exists = 'replace'):
    """"sends df to the table, schema, u"""
    df.to_sql(table, con = engine, schema = schema, if_exists = if_exists)
    print(f'Table {table} has been updated with to {schema} with {len(df)} rows')

In [ ]:
raw_data_to_db(df = users_clean1[users_clean1.user_id == 'Visitor'], table = 'unregistered_users_raw')

Table unregistered_users_raw has been updated with to bosh_marketing with 18907 rows


In [ ]:
raw_data_to_db(df = users_clean1[users_clean1.user_id != 'Visitor'], table = 'registered_users_raw')

Table registered_users_raw has been updated with to bosh_marketing with 14106 rows


In [ ]:
users_clean1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33013 entries, 3 to 32981
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   cookie_id           33013 non-null  object        
 1   session_id          33013 non-null  object        
 2   user_id             33013 non-null  object        
 3   geography           33013 non-null  object        
 4   email               33013 non-null  object        
 5   session_date        33013 non-null  datetime64[ns]
 6   channel             33013 non-null  object        
 7   landing_page_type   33013 non-null  object        
 8   final_page_type     33013 non-null  object        
 9   session_date_month  33013 non-null  int32         
 10  session_date_day    33013 non-null  int32         
 11  session_date_year   33013 non-null  int32         
 12  purchases           33013 non-null  int64         
 13  abandon_carts       33013 non-null  int64         


### Ecoystem marketing data model break down

• As previously discussed above our final merged makreting ecoystem data model granulartiy will be:

• dims: date, channel, landing_page_type (we can't split by plaform bc the there is other break down in the web data)

SELECT session_date, channel, landing_page_type

#### Metrics for final marketing ecosystem data model from web side

• visits : COUNT(DISTNCT session_id)

• unique_registered_user_visits = COUNT(DISTNCT CASE WHEN user_id != 'Visitor' THEN session_id END)

• unique_unregistered_user_visits = COUNT(DISTNCT CASE WHEN user_id == 'Visitor' then session_id END )

• unique_registered_users = COUNT(DISTNCT CASE WHEN user_id != 'Visitor' THEN user_id END)

• unique_unregistered_users = COUNT(DISTNCT CASE WHEN user_id == 'Visitor' then cookie_id END )

• unqiue_vistiors = COUNT(DISTNCT CASE WHEN user_id != 'Visitor' THEN user_id END) + COUNT(DISTNCT CASE WHEN user_id == 'Visitor' then cookie_id END)

• site_purchases = SUM(purchases)

• site_abandon_carts = SUM(abandon carts)

• registered_users_site_abandon_carts = SUM(CASE WHEN user_id != 'Visitor' THEN abandon_cart END)

• unregistered_users_site_abandon_carts SUM(CASE WHEN user_id == 'Visitor' THEN abandon_cart END)


Group by 1,2,3

• Marketing for analytics data will not be at the user or session level, usually at campaign level which this data set is misssing

• To create 

def send to df

In [ ]:
def clean_col_names(df):
    """renames all cols in df to be lower and no spaces"""
    df.columns = [i.lower().replace(' ', "_") for i in df.columns]
    return df